In [2]:

def graph_2(df, x, y):
    sns.set_theme(style='white')
    fig,ax=plt.subplots(1,figsize=SINGLE_PLOT_SIZE)
    sns.lineplot(
        data=df,
        x=x, y=y, 
        legend = False,
        estimator = None
    )
    #plt.axvline(2018, color="firebrick", linestyle='--',linewidth = 0.75)
    #plt.axvline(2015, color="darkseagreen", linestyle='--',linewidth = 0.75)
      # Background color
    fig.set_facecolor("white")
      # Y-axis
    current_values = plt.gca().get_yticks()
    plt.gca().set_yticklabels([round(x, 4) for x in current_values])
    plt.ylabel('', fontsize=14,loc='center',labelpad=10)

    # X-axis
    plt.xlabel('Year', fontsize=14,loc='center',labelpad=10)

    # Title
    #fig.suptitle(col_dict_titles[y], fontsize=14)
     
    fig.suptitle(y, fontsize=14)

    for year in year_of_change:
        plt.axvline(year, color="firebrick", linestyle='--', linewidth=0.75)
    # Box
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

      # Legend
    plt.legend(
        loc='lower center', 
        bbox_to_anchor=(0.5,-0.43),
    )
    fig.tight_layout()
    

def graph_3(df, x, y, categorical, values_cat):
    sns.set_theme(style='white')
    custom_colors = ['seagreen', 'steelblue', "darkseagreen", 'lightsteelblue','midnightblue','grey','goldenrod','burlywood','peru']
    fig,ax=plt.subplots(1,figsize=SINGLE_PLOT_SIZE)
    sns.lineplot(
        data=df,
        x=x, y=y, hue = categorical,
        palette = ['firebrick'] +list(islice(cycle(custom_colors), None, len(values_cat)-1)),
        legend = False,
        estimator = None
    )
    #plt.axvline(2018, color="firebrick", linestyle='--',linewidth = 0.75)
    #plt.axvline(2015, color="darkseagreen", linestyle='--',linewidth = 0.75)
      # Background color
    fig.set_facecolor("white")
      # Y-axis
    current_values = plt.gca().get_yticks()
    plt.gca().set_yticklabels([round(x, 4) for x in current_values])
    plt.ylabel('', fontsize=14,loc='center',labelpad=10)

    # X-axis
    plt.xlabel('Year', fontsize=14,loc='center',labelpad=10)

    # Title
    #fig.suptitle(col_dict_titles[y], fontsize=14)
     
    fig.suptitle(y, fontsize=14)

    for year in year_of_change:
        plt.axvline(year, color="firebrick", linestyle='--', linewidth=0.75)
    # Box
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

      # Legend
    plt.legend(
        loc='lower center', 
        bbox_to_anchor=(0.5,-0.43),
        ncol=3,
        labels = values_cat
    )
    fig.tight_layout()
    


In [3]:
df_theses = (spark.read.format('json').load('file:\\' + main_path+'theses-soutenues-ndjson.json'))

df_theses = (df_theses
            .select(*[col for col in df_theses.columns if col not in ['titres','sujets','resumes']],
                    *[func.col(col+'.en').alias(col + '_en') for col in ['titres','sujets','resumes'] ],
                    *[func.col(col+'.fr').alias(col + '_fr') for col in ['titres','sujets','resumes'] ]))

In [4]:
df_theses.printSchema()

root
 |-- accessible: string (nullable = true)
 |-- auteur: struct (nullable = true)
 |    |-- idref: string (nullable = true)
 |    |-- nom: string (nullable = true)
 |    |-- prenom: string (nullable = true)
 |-- cas: string (nullable = true)
 |-- code_etab: string (nullable = true)
 |-- date_soutenance: string (nullable = true)
 |-- directeurs_these: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- idref: string (nullable = true)
 |    |    |-- nom: string (nullable = true)
 |    |    |-- prenom: string (nullable = true)
 |-- discipline: string (nullable = true)
 |-- ecoles_doctorales: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- idref: string (nullable = true)
 |    |    |-- nom: string (nullable = true)
 |-- embargo: string (nullable = true)
 |-- etablissements_soutenance: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- idref: string (nullable = true)
 |    |    |

In [16]:
df_theses.select(func.explode(func.col('etablissements_soutenance')).alias('etablissements_soutenance')).distinct().sort('etablissements_soutenance.nom').count()

5656

In [5]:
scanR_path = 'D:\\scanR\\'
df_inst_scanR = spark.read.option("multiline","true").json('file:\\' + scanR_path + 'organizations.json')

In [6]:
df_scanR = spark.read.format('parquet').load('file:\\' + scanR_path + 'publications.parquet')
df_scanR.printSchema()

root
 |-- affiliations: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- affiliations: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- firstName: string (nullable = true)
 |    |    |-- fullName: string (nullable = true)
 |    |    |-- lastName: string (nullable = true)
 |    |    |-- person: string (nullable = true)
 |    |    |-- role: string (nullable = true)
 |    |    |-- rolePatent: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- role: string (nullable = true)
 |    |    |-- typeParticipant: string (nullable = true)
 |-- authorsCount: long (nullable = true)
 |-- doiUrl: string (nullable = true)
 |-- domains: array (nullable = true)
 |    |-- 

In [17]:
df_inst_scanR.printSchema()

root
 |-- acronym: struct (nullable = true)
 |    |-- default: string (nullable = true)
 |    |-- en: string (nullable = true)
 |    |-- fr: string (nullable = true)
 |-- activities: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- code: string (nullable = true)
 |    |    |-- label: struct (nullable = true)
 |    |    |    |-- default: string (nullable = true)
 |    |    |    |-- en: string (nullable = true)
 |    |    |    |-- fr: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- address: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- address: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- citycode: string (nullable = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- gps: struct (nullable = true)
 |    |    |    |-- lat: double (nullable = true)
 |    |    |    |-- lon: double (nullable = true)
 |    |    |-- localisationSu

In [12]:
(df_inst_scanR.withColumn('kind', func.explode(func.col('kind')))
 .withColumn('n', 1/func.count('kind').over(Window.partitionBy('id')))
.groupBy('kind')
 .agg(func.sum('n').alias('count')).sort("count", ascending = 0)).show()

+--------------------+------------------+
|                kind|             count|
+--------------------+------------------+
|       Secteur privé| 41334.83333333333|
|      Secteur public|14655.833333333334|
|Structure de rech...|11653.333333333334|
|Organisation inte...|              17.0|
+--------------------+------------------+



In [9]:
(df_inst_scanR
 .withColumn('externalIds', func.explode(func.col('externalIds')))
 .groupBy('id').agg(func.max((func.col('externalIds.type')=='idref').cast('int')).alias('has_id_ref'))
 .groupBy('has_id_ref').count()).show()

+----------+-----+
|has_id_ref|count|
+----------+-----+
|         1| 5286|
|         0|68886|
+----------+-----+



In [15]:
(df_inst_scanR
 .withColumn('externalIds', func.explode(func.col('externalIds')))
#.filter((func.col('externalIds.type')=='idref'))
 .select(func.col('id').alias('idref'), func.lit(1).alias('in_scanR'))
 .join(df_theses
       .select(func.explode(func.col('etablissements_soutenance')).alias('etablissements_soutenance'))
     .withColumn('idref', func.col('etablissements_soutenance.idref'))
       .distinct()
       .withColumn('in_theses', func.lit(1))
      , on = 'idref', how = 'inner')
).count()

0

In [12]:
test_label_doctoral_school = (df_theses
                              .withColumn('sujets_en', func.explode_outer(func.col('sujets_en')))
                              .withColumn('idref', func.explode_outer(func.col('ecoles_doctorales.idref')))
                              .groupBy(func.col('idref').alias('id_ecole_doc'))
                              .agg(func.collect_set(func.lower(func.col('sujets_en'))).alias('sujets_en'))
                             )
test_label_doctoral_school.show()

+------------+--------------------+
|id_ecole_doc|           sujets_en|
+------------+--------------------+
|   118096087|[diplomatic cerem...|
|   139076182|[regionalisation,...|
|   110349164|[god, china, inte...|
|   035586311|[china, camerooni...|
|   147297028|[surface acoustic...|
|   147964679|[crack initiation...|
|   059572175|[mlc901, optogene...|
|   149284713|[regionalisation,...|
|   167319809|[high-throughput ...|
|   14886662X|[vietnamese-frenc...|
|   149107854|[high-throughput ...|
|   152831738|[hereditament, mo...|
|   150495099|[casting, robotic...|
|   092177093|[multiobjective o...|
|   132250160|[m-csf, osteonecr...|
|   162253435|[sacrificial cris...|
|   194226271|                  []|
|   149627688|[satellite cells,...|
|   059079800|[crowdsource, ker...|
|   14912449X|[sublimation of s...|
+------------+--------------------+
only showing top 20 rows



In [17]:
test_label_doctoral_school.show()

ConnectionRefusedError: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée

In [25]:
(df_theses
 .select(func.explode(func.col('ecoles_doctorales')).alias('ecoles_doctorales'), func.col('discipline'), 'sujets_fr')
 .withColumn('sujets_fr', func.explode("sujets_fr"))
.groupBy('ecoles_doctorales').agg(func.flatten(func.array(func.collect_set('discipline'), func.collect_set('sujets_fr'))).alias('discipline'))
).limit(30).collect()

[Row(ecoles_doctorales=Row(idref=None, nom='Chimie, biologie, santé'), discipline=['Sciences du vivant', 'Pyrogallol', 'Endophytes', 'Superoxyde dismutase', 'OrbiTrap', 'QTOF']),
 Row(ecoles_doctorales=Row(idref=None, nom='Culture et Patrimoine'), discipline=["Sciences de l'information et de la communication", 'Patrimonialisation', 'Mediation', 'Patrimoine mondial', 'Construction patrimoniale', 'Arequipa, Pérou', 'Transformations urbaines', 'Patrimoine', 'Actualisation', 'Espagne', 'Patrimoine urbain', 'Architecture contemporaine']),
 Row(ecoles_doctorales=Row(idref=None, nom='Cultures Sociétés Territoires'), discipline=['Droit Privé', "Sciences de l'Information et de la Communication", "Histoire et civilisations: histoire des mondes modernes, histoire du monde contemporain, de l'art, de la musique", 'Droit Public', 'Langues et littératures anglaises et anglo saxones', 'Sciences économiques', 'Géographie', 'Sociologie', 'Langues et littératures romanes: espagnol, italien, portugais, au

In [18]:
df_theses.select('discipline').filter(func.size(func.split(func.col('discipline'), ' '))==1).distinct().show()

+--------------------+
|          discipline|
+--------------------+
|    Cristallographie|
|        Électronique|
|           Oncologie|
|               Qanse|
|Œnologie-ampélologie|
|              Hébreu|
|           Stratégie|
|             Ingénia|
|          Géographie|
|           Ergonomie|
|    Immunopathologie|
|         Entomologie|
|      Neurochirurgie|
|      Géostatistique|
|        Géomatériaux|
|     Géoarchitecture|
|        Systématique|
|     Chimie-Biologie|
|Vaisseaux-Hémosta...|
|        Informatique|
+--------------------+
only showing top 20 rows



In [10]:
(df_theses
.withColumn('ecoles_doctorales', func.explode(func.col('ecoles_doctorales')))
.select('ecoles_doctorales.idref','ecoles_doctorales.nom')
 .distinct()).limit(20).collect()

[Row(idref='139075909', nom='École doctorale Arts, Lettres, Langues (Rennes)'),
 Row(idref='153372885', nom='École doctorale des sciences de la vie, santé, agronomie, environnement (Clermont-Ferrand)'),
 Row(idref='149942109', nom='Ecole Doctorale Mecanique, Energetique, Genie Civil, Acoustique (MEGA) (Villeurbanne)'),
 Row(idref='076346102', nom='École doctorale Lettres, sciences humaines et sociales (Nice1992-2016)'),
 Row(idref='151367647', nom='École doctorale des sciences physiques et de l’ingénieur (Talence, Gironde)'),
 Row(idref='156498324', nom='École doctorale Humanités (Strasbourg ; 2009-....)'),
 Row(idref='162253435', nom='École doctorale Lettres, pensée, arts et histoire (Poitiers ; 2009-2018)'),
 Row(idref='149284357', nom='École doctorale sciences de gestion (Grenoble ; 1997-....)'),
 Row(idref='149107854', nom="École doctorale sciences et ingénierie des systèmes, de l'environnement et des organisations (Chambéry ; 2007-2021)"),
 Row(idref='230616127', nom='École doctor

In [5]:
df_theses.groupBy(func.col('resumes_en').isNull(), func.col('resumes_fr').isNull()).count().show()

+--------------------+--------------------+------+
|(resumes_en IS NULL)|(resumes_fr IS NULL)| count|
+--------------------+--------------------+------+
|                true|               false|121906|
|                true|                true| 61257|
|               false|               false|261767|
|               false|                true|   900|
+--------------------+--------------------+------+



In [6]:
#year_of_change = [2009]
#test_pd = (df_theses
#           .withColumn('annee_soutenance',func.substring(func.col('date_soutenance'),1,4).cast('int'))
#          .filter(func.col('annee_soutenance').between(2000,2020))
#          .groupBy('annee_soutenance','status')
#          .count()
#          ).toPandas()
#graph_3(test_pd, 'annee_soutenance','count','status', list(set(test_pd['discipline'])))

In [34]:
instits =  df_theses.select(func.explode_outer('etablissements_soutenance')).distinct()
instits.count()

5656

In [37]:
directeurs_these =  (df_theses
                     .withColumn('annee_soutenance',func.substring(func.col('date_soutenance'),1,4).cast('int'))
                      .filter(func.col('annee_soutenance').between(2000,2020))
                     .select(func.explode_outer('directeurs_these'))
                     .distinct())
directeurs_these.count()

90601

In [7]:
from unidecode import unidecode
textes = [unidecode(x[0]) for x in (df_theses
 .select(func.concat_ws(', ', func.col('titres_en'), #func.col('resumes_en'),
                        #func.col('discipline')
                       ).alias('txt'))
 .filter(func.col('txt').isNotNull())
 .distinct()).limit(20).collect()]
textes


['Dynamic coupling of electrochemical Impedance spectroscopy (EIS) with Raman spectroscopy for characterization of a conducting polymer film',
 'Ultrastructural and immunocytochemical approach of the pathogenic power of erwinia chrysanthemi: a new tool for the knowledge of the host cell wall',
 'Contribution to the analysis of hypermutation mechanisms of the variable regions of immunoglobulins',
 'Mechanical environnment simulation closed loop control of an electrodynamic shaker for transient testing',
 'Ponant islands, ilsets and archipelagoes from depopulated to overvisited areas work dealing with land-uses, management and conservation sinnce 1950',
 'Lepidochronological researches about posidonia oceanica (potamogetonaceae) : fluctuation of anatomical and morphological parameters of scales along rhizomes',
 'Iono-electronics: nonlinear alkali cation-induced complexation by substituted lutetium bisphthalocy anines',
 'Study of residual stress appearance in the process of on-line cons

In [8]:
!pip install sentencepiece
labels = ['History',
 'Political science',
 'Sociology',
 'Engineering',
 'Economics',
 'Art',
 'Materials science',
 'Environmental science',
 'Chemistry',
 'Biology',
 'Psychology',
 'Business',
 'Mathematics',
 'Physics',
 'Philosophy',
 'Geology',
 'Geography',
 'Medicine',
 'Computer science']
#translator = pipeline("translation", "Helsinki-NLP/opus-mt-fr-en")
for text in textes:
    #translated_text= translator(text)
    #print(f"Text: {text}", translated_text)
    #result = classifier(translated_text, labels)
    #print(f"Text: {text}")
    result = classifier(text, labels)
    print(text)
    print("Predicted Labels:")
    for label, score in zip(result["labels"], result["scores"]):
        print(f"- {label}: {score}")
    #print()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Dynamic coupling of electrochemical Impedance spectroscopy (EIS) with Raman spectroscopy for characterization of a conducting polymer film
Predicted Labels:
- Materials science: 0.05535513535141945
- Environmental science: 0.05506400391459465
- Political science: 0.05439143255352974
- Geology: 0.05414542555809021
- Computer science: 0.053667355328798294
- Engineering: 0.053290124982595444
- Art: 0.05317797511816025
- Physics: 0.052583497017621994
- Economics: 0.05258240923285484
- Chemistry: 0.05252118036150932
- History: 0.05215753987431526
- Mathematics: 0.05207093432545662
- Geography: 0.051945652812719345
- Sociology: 0.05187961831688881
- Business: 0.051642876118421555
- Philosophy: 0.05149030312895775
- Biology: 0.05110326036810875
- Psychology: 0.05103074759244919
- Medicine: 0.04990050196647644
Ultrastructural and immunocytochemical approach of the pathogenic power of erwinia chrysanthemi: a new tool for the knowledge of the host cell wall
Predicted Labels:
- Engineering: 0.053

Derivational morphology and diachrony : study of two slang works of the XVIth and XVIIth centuries : application to the contemporary complex lexicon
Predicted Labels:
- Geology: 0.056449249386787415
- Art: 0.05510524660348892
- History: 0.05499829724431038
- Materials science: 0.054885514080524445
- Political science: 0.05478041246533394
- Philosophy: 0.05388783663511276
- Computer science: 0.05318455025553703
- Geography: 0.05295197293162346
- Environmental science: 0.05256740003824234
- Psychology: 0.05225874483585358
- Physics: 0.05170523375272751
- Engineering: 0.05165622755885124
- Sociology: 0.05154820531606674
- Economics: 0.051420364528894424
- Mathematics: 0.051170703023672104
- Chemistry: 0.0508534200489521
- Biology: 0.0505770742893219
- Business: 0.05042734369635582
- Medicine: 0.04957219958305359
Molecular modelling study of entry HIV-1 gp120 and gp41 inhibition : application to the design of new potential antiretroviral agents
Predicted Labels:
- Art: 0.05612511187791824


In [18]:
result = classifier("", labels)
print("Predicted Labels:")
for label, score in zip(result["labels"], result["scores"]):
    print(f"- {label}: {score}")


Predicted Labels:
- Engineering: 0.05723641440272331
- History: 0.057059310376644135
- Sociology: 0.056443020701408386
- Art: 0.055829476565122604
- Chemistry: 0.05463247001171112
- Philosophy: 0.05402139201760292
- Physics: 0.053873930126428604
- Geography: 0.05365276709198952
- Medicine: 0.053359631448984146
- Psychology: 0.053082022815942764
- Biology: 0.052962176501750946
- Computer science: 0.05222296714782715
- Geology: 0.0521111935377121
- Mathematics: 0.05136319622397423
- Materials science: 0.049787312746047974
- Business: 0.04943038895726204
- Political science: 0.04859801381826401
- Economics: 0.04724447801709175
- Environmental science: 0.047089774161577225


In [ ]:
spark.catalog.clearCache()

In [ ]:
spark.stop()